# Creation Vector Store endpoint

In this notebook, I will create a databricks vector_store endpoint to proceed retrieval in the RAG 


In [0]:
%pip install --quiet -U   langchain==0.3.25 databricks-vectorsearch==0.60 
dbutils.library.restartPython()

## 1- Creation fo the needed functions for the vector store.

In [0]:
import time

def endpoint_exists(vsc, vs_endpoint_name):
  """
  Checks if the endpoint already exists.

    vsc is the Vector Search client.
    vs_endpoint_name is the name of the endpoint.

  """

  try:
    return vs_endpoint_name in [e['name'] for e in vsc.list_endpoints().get('endpoints', [])]
  except Exception as e:
    #Temp fix for potential REQUEST_LIMIT_EXCEEDED issue
    if "REQUEST_LIMIT_EXCEEDED" in str(e):
      print("WARN: couldn't get endpoint status due to REQUEST_LIMIT_EXCEEDED error. The demo will consider it exists")
      return True
    else:
      raise e

def wait_for_vs_endpoint_to_be_ready(vsc, vs_endpoint_name, n_attempts=10):
  """
  wait for the endpoint to be ready.

    vsc is the Vector Search client.
    vs_endpoint_name is the name of the endpoint.
    n_attempts is the number of attempts to wait for the endpoint to be ready, sleep_time is the pause duration

  """
  sleep_time = 100

  for i in range(n_attempts):
    try:
      endpoint = vsc.get_endpoint(vs_endpoint_name) # Does the endpoint already exist?
    except Exception as e:
      #Temp fix for potential REQUEST_LIMIT_EXCEEDED issue
      if "REQUEST_LIMIT_EXCEEDED" in str(e):
        print("WARN: couldn't get endpoint status due to REQUEST_LIMIT_EXCEEDED error. Please manually check your endpoint status")
        return
      else:
        raise e
    
    status = endpoint.get("endpoint_status", endpoint.get("status"))["state"].upper() #get status of the endpoint in upper case
    if "ONLINE" in status:
      return endpoint
    elif "PROVISIONING" in status or i <6:
      if i % 20 == 0: 
        print(f"Waiting for endpoint to be ready, this can take a few min... {endpoint}")
      time.sleep(sleep_time)
    else:
      raise Exception(f'''Error with the endpoint {vs_endpoint_name}. - this shouldn't happen: {endpoint}.\n Please delete it and re-run the previous cell: vsc.delete_endpoint("{vs_endpoint_name}")''')

  raise Exception(f"Timeout, your endpoint isn't ready yet: {vsc.get_endpoint(vs_endpoint_name)}")

In [0]:
def index_exists(vsc, vs_endpoint_name, vs_index_full_name):
    """
    Checks if the index already exists.

      vsc is the Vector Search client.
      vs_endpoint_name is the name of the endpoint.
      vs_index_full_name is the full name of the index.

    """
    try:
        vsc.get_index(vs_endpoint_name, vs_index_full_name).describe()
        return True
    except Exception as e:
        if 'RESOURCE_DOES_NOT_EXIST' not in str(e):
            print(f'Unexpected error describing the index. This could be a permission issue.')
            raise e
    return False
    
def wait_for_index_to_be_ready(vsc, vs_endpoint_name, vs_index_name, n_attempts=10):
  """
    wait for the index  to be ready.

    vsc is the Vector Search client.
    vs_endpoint_name is the name of the endpoint.
    vs_index_name is the name of the index.
    n_attempts is the number of attempts to wait for the endpoint to be ready, sleep_time is the pause duration

  """
  for i in range(n_attempts):
    idx = vsc.get_index(vs_endpoint_name, vs_index_name).describe()
    index_status = idx.get('status', idx.get('index_status', {}))
    status = index_status.get('detailed_state', index_status.get('status', 'UNKNOWN')).upper()
    url = index_status.get('index_url', index_status.get('url', 'UNKNOWN'))
    if "ONLINE" in status:
      return
    if "UNKNOWN" in status:
      print(f"Can't get the status - will assume index is ready {idx} - url: {url}")
      return
    elif "PROVISIONING" in status:
      if i % 40 == 0: print(f"Waiting for index to be ready, this can take a few min... {index_status} - pipeline url:{url}")
      time.sleep(100)
    else:
        raise Exception(f'''Error with the index - this shouldn't happen. DLT pipeline might have been killed.\n Please delete it and re-run the previous cell: vsc.delete_index("{vs_index_name}, {vs_endpoint_name}") \nIndex details: {idx}''')
  raise Exception(f"Timeout, your index isn't ready yet: {vsc.get_index(index_name, vs_endpoint_name)}")

def wait_for_model_serving_endpoint_to_be_ready(ep_name):
    from databricks.sdk import WorkspaceClient
    from databricks.sdk.service.serving import EndpointStateReady, EndpointStateConfigUpdate
    import time

    # Wait for it to be ready
    w = WorkspaceClient()
    state = ""
    for i in range(200):
        state = w.serving_endpoints.get(ep_name).state
        if state.config_update == EndpointStateConfigUpdate.IN_PROGRESS:
            if i % 40 == 0:
                print(f"Waiting for endpoint to deploy {ep_name}. Current state: {state}")
            time.sleep(10)
        elif state.ready == EndpointStateReady.READY:
          print('endpoint ready.')
          return
        else:
          break
    raise Exception(f"Couldn't start the endpoint, timeout, please check your endpoint for more details: {state}")

In [0]:
from databricks.vector_search.client import VectorSearchClient
VECTOR_SEARCH_ENDPOINT_NAME = "databricks_document_vs_endpoint_docling"
DOCUMENT_TABLE_NAME = "databricks_document_docling"
vsc = VectorSearchClient()


In [0]:
if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

In [0]:
catalog = "demo"
schema = "demo"
spark.sql(
  f"ALTER TABLE {catalog}.{schema}.{DOCUMENT_TABLE_NAME} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)"
)

In [0]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c

#The table we'd like to index
source_table_fullname = f"{catalog}.{schema}.{DOCUMENT_TABLE_NAME}"
# Where we want to store our index
vs_index_fullname = f"{catalog}.{schema}.{DOCUMENT_TABLE_NAME}_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  try:
    vsc.create_delta_sync_index(
      endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
      index_name=vs_index_fullname,
      source_table_name=source_table_fullname,
      pipeline_type="TRIGGERED",
      primary_key="id",
      embedding_source_column='contextualize_content', #The column containing our text
      embedding_model_endpoint_name='databricks-gte-large-en' #The embedding endpoint used to create the embeddings
    )
  except Exception as e:
    print(f"error : {e}  {VECTOR_SEARCH_ENDPOINT_NAME}")
    raise e
  #Let's wait for the index to be ready and all our embeddings to be created and indexed
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
else:
  #Trigger a sync to update our vs content with the new data saved in the table
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
  vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).sync()

print(f"index {vs_index_fullname} on table {source_table_fullname} is ready")